In [1]:
import os
import sys
import pandas
import numpy
import shutil
import cProfile, pstats, io

In [2]:
from algorithm_tester.tester_logic import run_algorithms_for_file
from algorithm_tester.tester_dataclasses import AlgTesterContext

In [3]:
def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [5]:
@profile
def profileRunAlgorithms(context, input_file):
    run_algorithms_for_file(context, input_file)

out_dir: str = ".tmp_out_dir"
with open("../data/Profile/Profile_40_inst.dat") as input_file:
    context: AlgTesterContext = AlgTesterContext(
        algorithms=["SA"], parser="KnapsackParser", communicators=[],
        check_time=True, time_retries=1, max_num=None,
        extra_options={"init_temperature":2500, "min_temperature":1, "cooling":0.995, "cycles":50},
        input_dir="../data/NK", output_dir=out_dir
        )
        
    profileRunAlgorithms(context, input_file)
shutil.rmtree(out_dir)

Running output for: Profile_40_inst_SA_sol.dat. Started 17:17:22 26.12.
         1258908 function calls in 1.139 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.139    1.139 <ipython-input-5-41e5329fd603>:1(profileRunAlgorithms)
        1    0.001    0.001    1.139    1.139 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.py:67(run_algorithms_for_file)
        2    0.000    0.000    1.133    0.566 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.py:34(get_instance_file_results)
        1    0.000    0.000    1.132    1.132 /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/timeit.py:162(timeit)
        1    0.000    0.000    1.132    1.132 <timeit-src>:2(inner)
        1    0.000    0.000    1.132    1.132 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_log